In [ ]:
%pip install pandas openpyxl transformers torch
%pip install llama-cpp-python

In [ ]:
from llama_cpp import Llama
from concurrent.futures import ThreadPoolExecutor

model_path = "/Users/pierr/Desktop/mistral-7b-instruct-v0.1.Q4_K_M.gguf"
max_context = 4096
llm = Llama(
    model_path,
    n_ctx=max_context
)

In [9]:
import json
import re

with open("metadata.json", "r", encoding="utf-8") as f:
    raw_metadata = json.load(f)

# Create a lookup table
metadata_lookup = {
    entry["code"]: f'{entry["name"]}. {entry["description"]}' 
    for entry in raw_metadata
}

def extract_codes(query):
    return re.findall(r"\[([A-Za-z0-9_]+)\]", query)

list_keywords = {"IN", "ANY", "NONE"}
null_keywords = {":NC": "Null type \"Not collected\"", ":NA": "Null type \"Not applicable\"", ":ND": "Null type \"Not disclosed\"", ":NI": "Null type \"No information\"", ":NM": "Null type \"Not meaningful\""}
def extract_nulls_and_lists(query):
    nulls = []
    lists = []
    contains_null = re.findall(r"\[([A-Za-z0-9_]+)\]\s+IS\s+(:[A-Z]{2})", query)
    for code, null_keyword in contains_null:
        nulls.append((code, null_keyword))
    
    contains_list = re.findall(r"\[([A-Za-z0-9_]+)\]\s+(IN|ANY|NONE)\b", query, re.IGNORECASE)
    for code, list_keyword in contains_list:
        lists.append((code, list_keyword.upper()))
    
    return nulls, lists

# Find the entry with the longest name + description combo
longest_entry = max(metadata_lookup.items(), key=lambda item: len(item[1]))

# Print the result
print(f'Longest entry code: {longest_entry[0]}')
print(f'Length: {len(longest_entry[1])}')
print(f'Content: {longest_entry[1]}')

Longest entry code: EUTaxManIntSerElcRevOverAlign
Length: 1353
Content: EU Taxon - Man, Instal, Serv of Elec Equip Overall Align. EU Taxon - Man, Instal, Serv of Elec Equip Overall Align: This factor identifies the overall alignment for Manufacture, installation, and servicing of high, medium and low voltage electrical equipment for electrical transmission and distribution that result in or enable a substantial contribution to climate change mitigation, covering the substantial contribution criteria, the do no significant harm criteria, and the minimum social safeguards. This is the aggregated result across all Taxonomy objectives. The possible values are: Aligned,Aligned (>90%),Aligned (>80%),Aligned (>70%),Aligned (>60%),Aligned (>50%),Aligned (>40%),Aligned (>30%),Aligned (>20%),Aligned (>10%),Aligned (>0%),Likely aligned (100%),Likely aligned (>90%),Likely aligned (>80%),Likely aligned (>70%),Likely aligned (>60%),Likely aligned (>50%),Likely aligned (>40%),Likely aligned (>30%),Li

In [5]:
def max_tokens(text, max_tokens=40):
    #moved this to use in below function
    tokens = llm.tokenize(text.encode("utf-8"))
    if len(tokens) <= max_tokens:
        return text
    truncated = llm.detokenize(tokens[:max_tokens]).decode("utf-8", errors="ignore")
    return truncated + "..."

def build_ordered_context(query, token_budget):
    #builds context in-order: null type definitions, enumerations for list types, and metadata lookups
    context_lines = []
    seen = set()
    used_tokens = 0

    null_hits, list_hits = extract_nulls_and_lists(query)
    codes_in_order = re.findall(r"\[([A-Za-z0-9_]+)\]", query)

    #tokenizer = llm.tokenizer
    for code in codes_in_order:
        if code in seen:
            continue
        seen.add(code)

        null_entry = next((kw for c, kw in null_hits if c == code), None)
        if null_entry:
            null_def = null_keywords.get(null_entry, f"No definition for {null_entry}")
            line = f"{code} = {null_entry} → {null_def}"
            tokens = len(llm.tokenize(line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(line)
            used_tokens += tokens

        base_meta = metadata_lookup.get(code, "No metadata found.")
        metadata_line = f"{code}: {max_tokens(base_meta, 100)}" 
        tokens = len(llm.tokenize(metadata_line.encode("utf-8")))
        if used_tokens + tokens > token_budget:
            break
        context_lines.append(metadata_line)
        used_tokens += tokens

        if any(c == code for c, _ in list_hits):
            enum_line = f"{code} (enumeration): {max_tokens(base_meta, 100)}"
            tokens = len(llm.tokenize(enum_line.encode("utf-8")))
            if used_tokens + tokens > token_budget:
                break
            context_lines.append(enum_line)
            used_tokens += tokens

    return "\n".join(context_lines)

def max_afforded_tokens(codes):
    return max(4096 // max(1, len(codes)), 100)


def translate_query(query, max_total_tokens=2048, max_output_tokens=256):
    codes = extract_codes(query)

    m_tokens = max_afforded_tokens(codes)

    # Initial prompt pieces
    instruction = "### Instruction:\nRephrase the following ESGish query into a concise natural English sentence. Each query is asking for all companies or issuers that match some paramater. Use the following metadata definitions for clarity:\n\n"
    query_part = f"\n\nQuery: {query}\n\n### Response:"

    # Tokenize instruction and query to calculate remaining token budget
    #tokenizer = llm.tokenize  # Built-in tokenizer
    instruction_tokens = len(llm.tokenize(instruction.encode("utf-8")))
    query_tokens = len(llm.tokenize(query_part.encode("utf-8")))
    token_budget = max_total_tokens - max_output_tokens - instruction_tokens - query_tokens

    # Build full context blocks for each code
    context = build_ordered_context(query, token_budget)
    print("query: ", query)
    print(context)

    """
    # Now iteratively add context blocks until budget is exhausted
    context = ""
    used_tokens = 0
    for block in context_blocks:
        block_tokens = len(tokenizer(block.encode("utf-8"))) + 1  # +1 for newline
        if used_tokens + block_tokens <= token_budget:
            context += block + "\n"
            used_tokens += block_tokens
        else:
            break  # stop once we're out of budget
    """
    # Final prompt
    prompt = instruction + context + query_part

    # Call model
    response = llm(prompt, max_tokens=max_output_tokens, temperature=0.1)
    return response["choices"][0]["text"].strip()




In [ ]:
import pandas as pd
from functools import partial
import time

#Reads and stores the Esgish queries
df = pd.read_excel("NullType_10Queries.xlsx")
queries = df["Esgish"].tolist()

#Ensures no overload and efficiency
batch_size = 100 
output_file = "translated_queries_TEST.xlsx"

#Looks at each query in each batch, calls the translate_query function, and stores it
for i in range(0, len(queries), batch_size):
    batch = queries[i:i + batch_size]
    translated_batch = []
    
    for query in batch:
        translated = translate_query(query)
        translated_batch.append(translated)
    
    df_batch = pd.DataFrame({
        "Esgish": batch,
        "English": translated_batch
    })

    #Makes a new file if needed, or adds onto the current file during each batch in case the program crashes at some point
    if i == 0:
        df_batch.to_excel(output_file, index=False)  
    else:
        with pd.ExcelWriter(output_file, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
            df_batch.to_excel(writer, index=False, header=False, startrow=i + 1)
    
    print(f"Saved batch {i // batch_size + 1} to {output_file}")

In [ ]:
import pandas as pd

output_file = "translated_queries_TEST5.xlsx"

# Extract the English column
df = pd.read_excel(output_file)
english_queries = df["English"].tolist()

# Function to request a comprehensibility rating from the model
def rate_comprehensibility(text, max_tokens=256):
    # Adjust the prompt to ask the model for a rating from 1 to 10
    prompt = f"""### Instruction:
Please rate the following text's comprehensibility from 1 to 10, where:
- 1 = Completely incomprehensible, nonsensical, or full of errors.
- 5 = Understandable with effort; some awkwardness, complexity, or minor errors.
- 10 = Perfectly clear, natural, and easy to understand.

Here are some examples:
Text: "asjdk asjd aksd" → Rating: 1
Text: "Provide list companies ESG data incomplete understandable" → Rating: 4
Text: "Please provide a list of companies with complete ESG data." → Rating: 9

Now, rate this text:
Text: {text}

### Response:"""
    # Call model (adjust temperature and other params as needed)
    response = llm(prompt, max_tokens=max_tokens, temperature=0.2)
    rating = response["choices"][0]["text"].strip()
    print(rating)
    # Ensure the response is a valid number between 1 and 10
    try:
        rating = int(rating)
        if 1 <= rating <= 10:
            return rating
    except ValueError:
        pass
    return None  # Return None if no valid rating is obtained

# List to store ratings
ratings = []

# Iterate through each English translation and get a rating
for query in english_queries:
    rating = rate_comprehensibility(query)
    ratings.append(rating)

# Add the ratings as a new column to the dataframe
df["Comprehensibility Rating"] = ratings

# Save the updated dataframe with ratings to a new Excel file
df.to_excel(output_file, index=False)

print("Comprehensibility ratings added and saved to 'translated_queries_with_ratings.xlsx'")


In [ ]:
# Run benchmark on a sample of queries
benchmark = TranslationBenchmark(llm, metadata_lookup, max_context=max_context)

# Load queries from CSV files
csv_files = ['res_len_from_1000_to_1200_len(10).csv',
                       'res_len_from_1200_to_1400_len(10).csv',
                       'res_len_from_1400_to_1600_len(10).csv',
                       'res_len_from_1600_to_1800_len(10).csv',
                       'res_len_from_1800_to_2000_len(10).csv']  # Add your CSV files here
queries = benchmark.load_queries_from_csv(csv_files)

# Use a subset of queries for benchmarking
sample_size = min(100, len(queries))  # Benchmark up to 100 queries
sample_queries = queries[:sample_size]

print(f"Running benchmark on {sample_size} queries...")
results = benchmark.run_benchmark(sample_queries)

print("\nBenchmark Results:")
print(f"Total Queries: {results['total_queries']}")
print(f"Success Rate: {results['success_rate']:.2%}")
print(f"Average Latency: {results['avg_latency']:.2f} seconds")
print(f"Latency Stats:")
print(f"  Min: {results['latency_stats']['min']:.2f}s")
print(f"  Max: {results['latency_stats']['max']:.2f}s")
print(f"  Median: {results['latency_stats']['median']:.2f}s")
print(f"  Std Dev: {results['latency_stats']['std']:.2f}s")

if results['error_types']:
    print("\nError Types:")
    for error, count in results['error_types'].items():
        print(f"  {error}: {count}")

# Generate visualizations
benchmark.plot_results("translation_benchmark_results(1).png")
benchmark.save_results("translation_benchmark_results(1).json")
benchmark.save_translations_to_csv("translated_queries(1).csv")

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import time
from typing import List, Dict, Any
import seaborn as sns
from datetime import datetime
import numpy as np

class TranslationBenchmark:
    def __init__(self, llm, metadata_lookup: Dict[str, str], max_context: int = None):
        self.llm = llm
        self.metadata_lookup = metadata_lookup
        self.max_context = max_context
        self.results = {
            'latencies': [],
            'file_names': [],
            'successes': [],
            'errors': []
        }
        
    def load_queries_from_csv(self, csv_files: List[str]) -> List[str]:
        """Load queries from multiple CSV files and track their source files."""
        all_queries = []
        for file in csv_files:
            df = pd.read_csv(file)
            queries = df['query_text'].tolist()
            all_queries.extend(queries)
            self.results['file_names'].extend([file] * len(queries))
        return all_queries

    def remove_outliers(self, data: List[float], threshold: float = 1.5) -> List[bool]:
        """
        Remove outliers using the IQR method.
        Returns a boolean mask where True indicates non-outlier values.
        """
        if not data:
            return []
        
        # Convert to numpy array for easier computation
        data_array = np.array(data)
        
        # Calculate Q1, Q3 and IQR
        Q1 = np.percentile(data_array, 25)
        Q3 = np.percentile(data_array, 75)
        IQR = Q3 - Q1
        
        # Define bounds
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        # Create mask for non-outliers
        mask = (data_array >= lower_bound) & (data_array <= upper_bound)
        return mask.tolist()

    def run_benchmark(self, queries: List[str], iterations: int = 5) -> Dict[str, Any]:
        """Run benchmark on a list of queries and track results by source file."""
        original_file_names = self.results['file_names'].copy()
        
        # Clear the results arrays
        self.results['latencies'] = []
        self.results['file_names'] = []
        self.results['successes'] = []
        self.results['errors'] = []
        
        for i, query in enumerate(queries):
            for _ in range(iterations):
                try:
                    start_time = time.time()
                    result = self.translate_query(query)
                    end_time = time.time()
                    
                    latency = end_time - start_time
                    self.results['latencies'].append(latency)
                    self.results['successes'].append(True)
                    self.results['errors'].append(None)
                    self.results['file_names'].append(original_file_names[i])
                    
                except Exception as e:
                    self.results['latencies'].append(None)
                    self.results['successes'].append(False)
                    self.results['errors'].append(str(e))
                    self.results['file_names'].append(original_file_names[i])
        
        return self._calculate_statistics()

    def _calculate_statistics(self) -> Dict[str, Any]:
        """Calculate statistics for the benchmark results with outlier removal."""
        # Filter out None values and get non-outlier mask
        latencies = np.array(self.results['latencies'])
        valid_mask = ~np.isnan(latencies)
        valid_latencies = latencies[valid_mask]
        
        if len(valid_latencies) > 0:
            # Get mask for non-outliers
            outlier_mask = self.remove_outliers(valid_latencies.tolist())
            clean_latencies = valid_latencies[outlier_mask]
        else:
            clean_latencies = []
        
        stats = {
            'total_queries': len(self.results['latencies']),
            'success_rate': sum(self.results['successes']) / len(self.results['successes']),
            'avg_latency': np.mean(clean_latencies) if len(clean_latencies) > 0 else 0,
            'latency_stats': {
                'min': np.min(clean_latencies) if len(clean_latencies) > 0 else 0,
                'max': np.max(clean_latencies) if len(clean_latencies) > 0 else 0,
                'median': np.median(clean_latencies) if len(clean_latencies) > 0 else 0,
                'std': np.std(clean_latencies) if len(clean_latencies) > 0 else 0,
                'outliers_removed': len(valid_latencies) - len(clean_latencies) if len(valid_latencies) > 0 else 0
            },
            'error_types': self._count_error_types()
        }
        return stats

    def _count_error_types(self) -> Dict[str, int]:
        """Count occurrences of each error type."""
        error_counts = {}
        for error in self.results['errors']:
            if error:
                error_counts[error] = error_counts.get(error, 0) + 1
        return error_counts

    def plot_results(self, output_file: str = None):
        """Create visualizations of the benchmark results with outlier handling."""
        plt.figure(figsize=(15, 10))
        
        # Create DataFrame
        df = pd.DataFrame({
            'File': self.results['file_names'],
            'Latency (secs)': self.results['latencies'],
            'Success': self.results['successes']
        })
        
        # Remove outliers for plotting
        df_clean = df.copy()
        df_clean['is_outlier'] = False
        
        for file in df['File'].unique():
            file_mask = df['File'] == file
            latencies = df.loc[file_mask, 'Latency (secs)'].dropna()
            if len(latencies) > 0:
                outlier_mask = self.remove_outliers(latencies.tolist())
                df_clean.loc[file_mask, 'is_outlier'] = ~pd.Series(outlier_mask, index=latencies.index)
        
        # Plot 1: Box plot of latencies by file (with outliers)
        plt.subplot(2, 2, 1)
        sns.boxplot(data=df, x='File', y='Latency (secs)', showfliers=True)
        plt.xticks(rotation=90)
        plt.title('Latency Distribution by File (with outliers)')
        
        # Plot 2: Box plot of latencies by file (without outliers)
        plt.subplot(2, 2, 2)
        sns.boxplot(data=df_clean[~df_clean['is_outlier']], x='File', y='Latency (secs)')
        plt.xticks(rotation=90)
        plt.title('Latency Distribution by File (without outliers)')
        
        # Plot 3: Latency over time (without outliers)
        plt.subplot(2, 2, 3)
        plt.scatter(df_clean[~df_clean['is_outlier']].index, 
                   df_clean[~df_clean['is_outlier']]['Latency (secs)'], 
                   alpha=0.7, s=50)
        plt.title('Latency Over Time (without outliers)')
        plt.xlabel('Query Index')
        plt.ylabel('Latency (seconds)')
        
        # Plot 4: Outlier distribution
        plt.subplot(2, 2, 4)
        outlier_counts = df_clean.groupby('File')['is_outlier'].sum()
        outlier_counts.plot(kind='bar')
        plt.title('Number of Outliers by File')
        plt.xticks(rotation=90)
        plt.ylabel('Number of Outliers')
        
        plt.tight_layout()
        
        if output_file:
            plt.savefig(output_file)
        plt.show()

    def save_results(self, output_file: str):
        """Save benchmark results to a JSON file."""
        results = {
            'statistics': self._calculate_statistics(),
            'raw_results': {
                'latencies': self.results['latencies'],
                'file_names': self.results['file_names'],
                'successes': self.results['successes'],
                'errors': self.results['errors']
            }
        }
        
        with open(output_file, 'w') as f:
            json.dump(results, f, indent=4)

    def save_translations_to_csv(self, output_file: str):
        """Save translations and their metadata to a CSV file."""
        df = pd.DataFrame({
            'File': self.results['file_names'],
            'Latency (secs)': self.results['latencies'],
            'Success': self.results['successes'],
            'Error': self.results['errors']
        })
        df.to_csv(output_file, index=False)

    def translate_query(self, query: str):
        """Example translate_query function. Replace with your actual LLM call."""
        time.sleep(0.1)  # Simulating a 100 ms translation time
        return f"Translated: {query}"  # Simulated translation result